# 0. はじめに

## そもそもアプリがユーザーにどの程度効果を与えているかを知るための解析

### 

# 1. 準備

### 1.1. 基本設定

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns #seabornない人はpip installしてね
import os
from datetime import datetime
import numpy as np
from pathlib import Path

# カレントディレクトリを.pyと合わせるために以下を実行
if Path.cwd().name == "notebook":
    os.chdir("..")

# 設定
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 500)
pd.set_option('display.max_columns', 500)

# 浮動小数点数を小数点以下3桁で表示するように設定
pd.set_option('display.float_format', '{:.3f}'.format)

In [ ]:
# Mac Matplotlibのデフォルトフォントをヒラギノ角ゴシックに設定
plt.rcParams['font.family'] = 'Hiragino Sans'

In [ ]:
# Windows MatplotlibのデフォルトフォントをMeiryoに設定
plt.rcParams['font.family'] = 'Meiryo'

### 1.2. csv読み込み

In [ ]:
# point_history.csvの読み込み
df_point_history_sorce = pd.read_csv('data/input/point_history_cleansing.csv')

In [ ]:
# ユーザー基本情報の読み込み
df_user_base_sorce = pd.read_csv("data/input/ユーザー基本情報_2023-12-21.csv", encoding="shift-jis")

### 1.3. データクレンジング

#### 1.3.1. df_user_base(ユーザ基本情報)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_user_base = df_user_base_sorce.copy()

# 紛らわしい列名を改名
df_user_base = df_user_base.rename(columns={'登録日時': 'アプリ登録日時', '最終利用日': 'アプリ最終利用日'})

# objectをdatetimeに変更
df_user_base['アプリ登録日時'] = pd.to_datetime(df_user_base['アプリ登録日時'], errors='coerce')
df_user_base['アプリ最終利用日'] = pd.to_datetime(df_user_base['アプリ最終利用日'], errors='coerce')
df_user_base['birthday'] = pd.to_datetime(df_user_base['birthday'], errors='coerce')

# 6歳未満(1543個)と100歳以上(12個)を削除
df_user_base = df_user_base[ (df_user_base['birthday'] < pd.to_datetime('2017-01-01')) & (df_user_base['birthday'] > pd.to_datetime('1924-01-01'))]

# df_user_baseに"age"と"age_group"のカラムを追加
df_user_base['age'] = pd.Timestamp.now().year - df_user_base['birthday'].dt.year    # ageの算出・追加

# 年代の算出・追加
bins = [0, 20, 30, 40, 50, 60, 70, 80, 90, 100]
labels = ['0-20', '21-30', '31-40', '41-50', '51-60', '61-70', '71-80', '81-90', '91-100']
df_user_base['age_group'] = pd.cut(df_user_base['age'], bins=bins, labels=labels, right=False)

# 今回使用しない可能性が高いカラムは削除
df_user_base = df_user_base.drop(['登録日', 'カード種類', 'スーパー', '都道府県', '市区町村', '登録店舗', 'カード登録日', 'カード更新日', 'birthday'], axis=1)
df_user_base = df_user_base.drop_duplicates(subset='利用者ID', keep='first')
df_user_base = df_user_base.sort_values(by='アプリ登録日時')
df_user_base.head()
# len(df_user_base)

#### 1.3.2. df_point_history(point_history.csv)のクレンジング

In [ ]:
# DataFrameのコピーを作成
df_point_history = df_point_history_sorce.copy()

# objectをdatetimeに変更
df_point_history['use_date'] = pd.to_datetime(df_point_history['use_date'], errors='coerce')

# 今回使用しない可能性が高いカラムは削除
df_point_history = df_point_history.drop([
    'Unnamed: 0',
    'id',
    'series_id',
    'shop_id',
    'shop_name',
    'card_id',
    'リサイクル分類ID',
    'amount',
    'created_at',
    'updated_at',
    '支店ID',
    'super',
    'shop_name_1',
    'shop_id_1',
    'updated_at_1',
    'store_opening_time',
    'store_closing_time',
    'created_at_1',
    'rps_opening_time',
    'rps_closing_time',
    'store_latitude',
    'store_longitude',
    'total_amount',], axis=1)
df_point_history = df_point_history.sort_values(by='use_date')
df_point_history.head(1)

In [ ]:
#df_point_history[df_point_history['user_id']==1138]

#### 1.3.3. 分析に必要なカラムの作成

アプリ継続利用期間（アプリ最終利用日-アプリ登録日時）

In [ ]:
# 継続利用期間を計算
df_user_base['アプリ継続利用期間(月)'] = (df_user_base['アプリ最終利用日'] - df_user_base['アプリ登録日時']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['アプリ継続利用期間(月)'] >= 0]
df_user_base.head()
# len(df_user_base[df_user_base['継続利用期間(月)'] == 0])

継続利用期間（point_historyのuse_date列からRPS最終利用日を抽出したver.）　231228 norosen

In [ ]:
## 実行に10分ほどかかります。。
# 利用者id を抽出
user_id_all = df_user_base['利用者ID'].values

# 各利用者id に対して「RPS利用開始日」「RPS最終利用日」を抽出
first_use_RPS = []
last_use_RPS = []

for uid in user_id_all:

    df_one_user = df_point_history[df_point_history['user_id']==uid]
    
    if len(df_one_user) != 0:

        first_use_RPS.append(df_one_user['use_date'].iloc[0])
        
        last_use_RPS.append(df_one_user['use_date'].iloc[-1])
        
    else:

        first_use_RPS.append('NaT')
        
        last_use_RPS.append('NaT')

In [ ]:
# 「RPS利用開始日」「RPS最終利用日」カラムを作成
df_user_base['RPS利用開始日'] = first_use_RPS
df_user_base['RPS最終利用日'] = last_use_RPS

df_user_base['RPS利用開始日'] = pd.to_datetime(df_user_base['RPS利用開始日'], errors='coerce')
df_user_base['RPS最終利用日'] = pd.to_datetime(df_user_base['RPS最終利用日'], errors='coerce')
df_user_base.head()

In [ ]:
# RPS継続利用期間を計算
df_user_base['RPS継続利用期間(月)'] = (df_user_base['RPS最終利用日'] - df_user_base['RPS利用開始日']).dt.days / 30  # 月単位で計算
df_user_base = df_user_base[df_user_base['RPS継続利用期間(月)'] >= 0]
df_user_base.head()

In [ ]:
# 列の順番を分かりやすいように入れ替える
order_df_user_base = ['利用者ID', 'カード登録', '現在ランク', 'age', 'age_group',
                      'アプリ登録日時', 'アプリ最終利用日', 'アプリ継続利用期間(月)',                   
                      'RPS利用開始日', 'RPS最終利用日', 'RPS継続利用期間(月)']

df_user_base = df_user_base[order_df_user_base]

In [ ]:
# RPS列追加に10分くらいかかるので、できあがったものをcsvとして出力しておく
df_user_base.to_csv("data/input/ユーザー基本情報_2023-12-21_RPS利用日追加.csv", encoding='shift-jis')

In [ ]:
n=9150
display(df_user_base[df_user_base['利用者ID']==n])
print((df_user_base[df_user_base['利用者ID']==n]['アプリ登録日時'] - df_user_base[df_user_base['利用者ID']==n]['RPS利用開始日']).dt.days)

display(df_point_history[df_point_history['user_id']==n])

#df_test = df_point_history[df_point_history['user_id']==n]
#plt.plot(df_test['use_date'], df_test['amount_kg'])
#plt.xticks
#plt.show()

In [ ]:
# アプリ登録年月ごとに集計
df_user_base['アプリ登録年月'] = df_user_base['アプリ登録日時'].dt.strftime("%Y/%m")

n_users_registered = df_user_base['アプリ登録年月'].value_counts().sort_index()
plt.plot(n_users_registered)

plt.title('月ごとのアプリ登録者数')
plt.xticks(rotation=90, fontsize=8)
plt.ylabel('アプリ登録者数')
plt.grid(True)
plt.show()

In [ ]:
# 累積データの計算
cumulative_data = np.cumsum(df_user_base['アプリ登録年月'].value_counts().sort_index().values)

plt.plot(df_user_base['アプリ登録年月'].value_counts().sort_index().index, cumulative_data)

plt.title('月ごとのアプリ登録者数（累積）')
plt.xticks(rotation=90, fontsize=8)
plt.ylabel('アプリ登録者数')
plt.grid(True)
plt.show()

In [ ]:
# RPS開始日を年月ごとに集計
df_user_base['RPS開始年月'] = df_user_base['RPS利用開始日'].dt.strftime("%Y/%m")

n_users_registered = df_user_base['RPS開始年月'].value_counts().sort_index()
plt.plot(n_users_registered)
plt.xticks(rotation=90, fontsize=8)
plt.grid(True)
plt.show()

In [ ]:
# アプリを継続利用した「Yes, No」、RPSを継続利用した「Yes, No」の4通りの割合を算出する関数
# Matrixとして可視化
# Threshold : 月を入力

def matrix_usage(df_user_base_ym, month_threshold):
    '''
    アプリを継続利用した「Yes, No」、RPSを継続利用した「Yes, No」の4通りの割合を算出する関数
    
    Params
    ------
    df_user_base_ym : y年m月にぐるっとポンに登録したユーザー
    month_threshold : 継続利用したとみなすための閾値（単位：月）
    
    Return
    ------
    [n_1_1, n_0_1, n_1_0, n_0_0] : 割合の算出結果
    
    '''

    df_user_base_ym_1_1 = df_user_base_ym[(df_user_base_ym['アプリ継続利用期間(月)'] >= month_threshold) & (df_user_base_ym['RPS継続利用期間(月)'] >= month_threshold)]
    df_user_base_ym_0_1 = df_user_base_ym[(df_user_base_ym['アプリ継続利用期間(月)'] < month_threshold) & (df_user_base_ym['RPS継続利用期間(月)'] >= month_threshold)]
    df_user_base_ym_1_0 = df_user_base_ym[(df_user_base_ym['アプリ継続利用期間(月)'] >= month_threshold) & (df_user_base_ym['RPS継続利用期間(月)'] < month_threshold)]
    df_user_base_ym_0_0 = df_user_base_ym[(df_user_base_ym['アプリ継続利用期間(月)'] < month_threshold) & (df_user_base_ym['RPS継続利用期間(月)'] < month_threshold)]


    n_1_1 = len(df_user_base_ym_1_1) / len(df_user_base_ym)
    n_0_1 = len(df_user_base_ym_0_1) / len(df_user_base_ym)
    n_1_0 = len(df_user_base_ym_1_0) / len(df_user_base_ym)
    n_0_0 = len(df_user_base_ym_0_0) / len(df_user_base_ym)

    mat_columns = ['アプリを' + str(month_threshold) + 'か月以上継続利用', 'アプリを' + str(month_threshold) + 'か月以内に利用終了']
    mat_index = ['RPSを' + str(month_threshold) + 'か月以上継続利用', 'RPSを' + str(month_threshold) + 'か月以内に利用終了']
    mat_usage = pd.DataFrame([[n_1_1, n_0_1], [n_1_0, n_0_0]], columns=mat_columns, index=mat_index)

    #display(mat_usage)
    
    return [n_1_1, n_0_1, n_1_0, n_0_0]


In [ ]:
# 2021年12月にアプリを登録したユーザーに絞って解析
year_month = datetime(2021,12,1).strftime("%Y/%m")

df_user_base_year_month = df_user_base[df_user_base['アプリ登録年月']==year_month]

# RPSの開始日の方が先のユーザーは別で解析するので分ける
df_user_base_year_month_app_derived = df_user_base_year_month[(df_user_base['アプリ登録日時'] - df_user_base_year_month['RPS利用開始日']).dt.days <0]  # 240107修正: >=0 から <0 に
df_user_base_year_month_RPS_derived = df_user_base_year_month[(df_user_base['アプリ登録日時'] - df_user_base_year_month['RPS利用開始日']).dt.days >=0]   # 240107修正: <0 から >=0 に

In [ ]:
print(len(df_user_base_year_month_app_derived))
print(len(df_user_base_year_month_RPS_derived))

In [ ]:
# アプリ、RPSの継続利用期間を比較
df_analyzed = df_user_base_year_month_app_derived.copy()  # .copy（）は必ずつけること

bins_duration = np.arange(27)
plt.hist(df_analyzed['アプリ継続利用期間(月)'], bins=bins_duration, label='アプリ', alpha=0.7)
plt.hist(df_analyzed['RPS継続利用期間(月)'], bins=bins_duration, label=' RPS', alpha=0.7)

plt.title('アプリ、RPSの継続利用期間\n（' + year_month + 'にアプリを登録したユーザー）')
plt.xlabel('継続利用期間（月）')
plt.ylabel('Counts')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
#plt.ylim([0,100])
plt.show()

・継続利用期間は「始めてから何か月で利用を辞めたか」と言い換えられる  

比較  
・対照的なヒストグラム  
・アプリは最初に辞める人が多いのに対して、RPSは継続利用する人が多い。中盤（5~20か月）は同じような値。
 

考察  
・極端な話ぐるっとポンが無くても皆継続利用してくれる  
・ユーザーにとって、現状ぐるっとポンは利用し始めるための障壁が高い？  
・ぐるっとポンのインストールを目的としたキャンペーンの実施によって、新規RPS利用者を獲得。その後はぐるっとポンを使用せずRPSだけを継続利用してる層がマジョリティー？

In [ ]:
# アプリ利用、RPS利用の推移を可視化
df_analyzed = df_user_base_year_month_app_derived.copy()  # .copy（）は必ずつけること

month_thre_all = [0.03, 3, 6, 12, 18, 22]
dict_mat_usage = {}

for m_thre in month_thre_all:
    
    dict_mat_usage[m_thre] = matrix_usage(df_analyzed, m_thre)
    
    
# 積み上げ棒グラフの描画
bottom = np.zeros(len(month_thre_all))

label_usage = ['アプリ:Yes, RPS:Yes', 'アプリ:No,  RPS:Yes',
               'アプリ:Yes, RPS:No', 'アプリ:No,  RPS:No']

for i, l_usage in enumerate(label_usage):
        
    values = [dict_mat_usage[m_thre][i] for m_thre in month_thre_all]
    
    plt.bar(month_thre_all, values, bottom=bottom, label=l_usage, width=1, zorder=i+2)

    bottom += values
    
   # 各棒の上にテキストを追加
for m_thre in month_thre_all:

    plt.text(m_thre, 1.0, str(m_thre), ha='center', va='bottom', c='green')

# 凡例を表示
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)

# グラフのタイトルと軸ラベルを設定
plt.title('アプリ利用 & RPS利用の推移\n（' + year_month + 'にアプリを登録したユーザー）')
plt.xlabel("月（month）", c='green')
plt.ylabel("割合")
plt.ylim([0, 1.1])
plt.grid(True, axis='y', zorder=1, alpha=0.3)

plt.show()

# 240107 古紙投入量予測AI
### LSTM or Transformer を用いてモデル構築
### データ：2021/12 にアプリを登録し、RPSを使い始めたユーザー 約3000人


### 1. AI構築の前段階として、以下の項目についてチェック
・month vs. 古紙投入量/月 をプロットし、全体を可視化モデル分ける必要ありそうかを判断  

### 2. モデル学習：半年くらいまでのデータを入力し、次の月の古紙投入量を出力とする（暫定）
・データは2021/12から2023/5（18か月後）までを扱う

In [ ]:
# 利用者id を抽出
user_id_analyzed = df_analyzed_user_base['利用者ID'].values

# 抽出した利用者idに対応したuser_idの行だけ抽出
df_analyzed_point_history = df_point_history[df_point_history['user_id'].isin(user_id_analyzed)]
df_analyzed_point_history = df_analyzed_point_history[df_analyzed_point_history['status']==1]
df_analyzed_point_history['use_month'] = df_analyzed_point_history['use_date'].dt.strftime("%Y/%m/")

In [ ]:
display(df_analyzed_point_history.tail())

In [ ]:
# ユーザーごとに各月の古紙投入量を算出・集計
group_uid_umonth = df_analyzed_point_history.groupby(['user_id', 'use_month'])['amount_kg'].mean()
group_uid_umonth

In [ ]:
# ToDo: 各ユーザーに対して、投入量が0の月に0を入力する。→ 学習・検証用データを.csvとしてまとめる

In [ ]:
uid_all_p_history = df_analyzed_point_history['user_id'].unique()
uid_all_p_history.sort()
print(uid_all_p_history)
for uid in uid_all_p_history[400:420]:
    
    plt.plot(pd.to_datetime(group_uid_umonth.loc[uid].index, format='%Y/%m'), group_uid_umonth.loc[uid], alpha=0.3)

plt.xticks(rotation=90, fontsize=8)
plt.axhline(15, c='red', linestyle='--')
#plt.ylim([0,50])
plt.grid(True)
plt.show()

In [ ]:
# RPSを使ってる人のうち、アプリを使っている人と使っていない人の比較
month_threshold = 1

df_user_base_year_month_1_1 = df_user_base_year_month[(df_user_base_year_month['アプリ継続利用期間(月)'] >= month_threshold) & (df_user_base_year_month['RPS継続利用期間(月)'] >= month_threshold)]
df_user_base_year_month_0_1 = df_user_base_year_month[(df_user_base_year_month['アプリ継続利用期間(月)'] < month_threshold) & (df_user_base_year_month['RPS継続利用期間(月)'] >= month_threshold)]

#display(df_user_base_year_month_0_1.head())

# RPS継続利用期間
plt.hist(df_user_base_year_month_1_1['RPS継続利用期間(月)'], bins=30, label='Yes-Yes', alpha=0.3)
plt.hist(df_user_base_year_month_0_1['RPS継続利用期間(月)'], bins=30, label=' No-Yes', alpha=0.3)

plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0)
plt.ylim([0,100])
plt.show()

In [ ]:
# ユーザーの古紙投入の間隔ってどれくらい？

#### ※df_user_baseとdf_user_basef_point_historyをマージ

In [ ]:
# point_history.csvとユーザー基本情報_2023-12-21.csvをマージ
df_merge = pd.merge(df_point_history, df_user_base, left_on='user_id', right_on='利用者ID', how='left')

In [ ]:
df_merge

# 2. 分析

# 分析1
 【ランクごとの継続利用期間（最終更新日-利用開始日）の比較】   
 ＜目的＞15kg/月以上出し続ける能力がある人の方が、ない人より継続利用期間が長いことを証明<br>
　＜方法＞シルバー以上の人と、ブロンズの人の継続利用期間の比較（中央値）<br>

 ＜結果＞ランクが上がるにつれて、中央値は上昇傾向  
 ＜懸念＞データが怪しい。特に継続利用期間の最少が間違っていそう（ゴールドとか）<br>
     　　　　→クレンジングの再見直し

In [ ]:
# ボックスプロットでランク別に継続利用期間を表示
sns.boxplot(x='現在ランク', y='継続利用期間(月)', data=df_user_base, order=['ブロンズ', 'シルバー', 'ゴールド', 'プラチナ'])
plt.xlabel('現在ランク')
plt.ylabel('継続利用期間(月)')
plt.title('ランク別継続利用期間の比較')
plt.show()


In [ ]:
for rank in ['ブロンズ', 'シルバー', 'ゴールド', 'プラチナ']:
    median_duration = df_merge_rank_positive[df_merge_rank_positive['現在ランク'] == rank]['継続利用期間'].median()
    print(f"{rank}ランクの継続利用期間の中央値: {median_duration}ヶ月")

In [ ]:
# ヒストグラムで分布を表示
plt.figure(figsize=(10, 6))
sns.histplot(df_merge_rank_positive, x='継続利用期間', hue='現在ランク', element='step', stat='density', common_norm=False)
plt.xlabel('継続利用期間（月）')
plt.ylabel('密度')
plt.title('シルバーとゴールドランクの継続利用期間の分布')
plt.show()

# KDEプロットで分布を表示
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_merge_rank_positive, x='継続利用期間', hue='現在ランク', common_norm=False)
plt.xlabel('継続利用期間（月）')
plt.ylabel('密度')
plt.title('シルバーとゴールドランクの継続利用期間の分布（KDE）')
plt.show()

# 分析4：年代別のrankの割合

## rankはどの時点のものにする？
## 棒グラフで可視化

# (参考) visualization_point_history.ipynb の分析

## 2-1. rankごとの平均年間持込量kgの算出

・rank_id毎のrecycle_amount_per_year平均の算出  
・rankが高いほど、年間持ち込み量が多いという仮説を証明する(当然？)  
・rank_idは、0:ブロンズ、1:シルバー、2:ゴールド、3:プラチナ  
・1か月に15kg持ち込み続けるとランクUP。ランクダウンは不明(1か月15kgを達成しないとダウン説、、？)  
※以下の手順を踏むと「rank毎の1回の持ち込み量平均」になってしまうので注意  
　　① amount_kgは1回の持ち込み量。これをrank_id毎にsumする  
　　② rank_idそれぞれの出現回数をsumする  
　　③ ①を②で割る  

In [ ]:
# まずそれぞれのカラムの基本統計量を確認しておく
print('recycle_amount_per_year')
print(df_merge['recycle_amount_per_year'].describe())
print('------------------')
print('rank_id')
print(df_merge['rank_id'].describe())
print('------------------')
print('rank_idのユニーク')
print(df_merge['rank_id'].unique())

※rankは3（プラチナ）が不在な点に注意

In [ ]:
# user_idの重複行を削除（非効率だと思いつつ、、、）
df_merge_duplicates_user_id = df_merge.drop_duplicates(subset='user_id')

ランク毎の持ち込みamountの平均 = df_merge.groupby('rank_id')['recycle_amount_per_year'].mean()
ランク毎の持ち込みamountの平均

### 得られた示唆

・ランクが高いほど、平均年間持ち込み量は多い  
　→「持ち込み量が多いから、ランクが上がった」という解釈の方が正しい？  
 ・rankのカラムのみ、user_infoに紐づけておくと、user_infoの情報量が増えてよいかも


## 2-2. ユーザ個人に着目して指針を得る 

### 2-2-1. トップユーザのamount_kg推移

・point、coin、rankが、ユーザの持ち込み量に影響を与えているのか確認  
・rankを維持したい人


In [ ]:
# 'user_id'の各値の出現回数を計算
user_id_counts = df_merge['user_id'].value_counts().reset_index()
user_id_counts.columns = ['user_id', 'count']

# 元のDataFrameにcountをマージ
df_merge = df_merge.merge(user_id_counts, on='user_id')

# 出現回数に基づいてソート（降順）
df_merge = df_merge.sort_values(by='count', ascending=False)

In [ ]:
# 重複を削除して、上位100名のcountをtableで俯瞰する
df_merged_duplicates_user_id = df_merge.drop_duplicates(subset='user_id')
top_100 = df_merged_duplicates_user_id.head(100)
top_100

In [ ]:
# top_100の年代別人数を表示
sns.histplot(data=top_100, x='age_group')

plt.title('Age Band Distribution')
plt.xlabel('Age Band')
plt.ylabel('Frequency')

plt.show()

In [ ]:
# 特定の人のamount_kgの推移を確認
# 特定の人のデータのみ抽出
target_user_id = 1152
df_merge_target = df_merge[df_merge['user_id'] == target_user_id]
df_merge_target = df_merge_target.sort_values(by='use_date', ascending=True)

df_merge_target['use_date'] = pd.to_datetime(df_merge_target['use_date'])

# use_dateを月ごとに集約（データ数が多すぎて折れ線グラフが見づらい）
df_merge_target_monthly= df_merge_target[['use_date', 'amount_kg']].resample('M', on='use_date').sum()

display(df_merge_target_monthly)

# 集約されたデータで折れ線グラフを作成
plt.figure(figsize=(10, 6))
plt.plot(df_merge_target_monthly.index, df_merge_target_monthly['amount_kg'], marker='o')
plt.title('Monthly Average of Amount (kg)')
plt.xlabel('Month')
plt.ylabel('Average Amount (kg)')

# データをすべて使って折れ線グラフを作成
plt.figure(figsize=(10, 6))
plt.plot(df_merge_target_monthly['use_date'], df_merge_target_monthly['rank_id'], marker='o')
plt.title('rank')
plt.xlabel('date')
plt.ylabel('rank')
plt.show()

# coinが付与され始めた時期のデータを表示
filtered_df  = df_merge_target[df_merge_target['coin'] > 0]
filtered_df.head()


### まとめ

・15kgを達成しても、すぐにランクが上がらない矛盾あり  
・15kgを未達でもすぐにランクが下がらない矛盾あり  
・アプリを利用していなくても履歴は残る。アプリを利用しているか否かは'coin'(付与コイン)で判断する  

## 2-3. rankシステムの効果測定

・rank_idのレコード

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].mean().reset_index()
# avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = avg_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
rank_differences.head(100)

### 1,2,3ごとの平均値を出してみる

In [ ]:
average_difference_rank = rank_differences.mean(skipna=True)
average_difference_rank

### 1,2,3ごとの合計を出してみる

In [ ]:
# TODO: amount_kgの平均を出してしまっているので、user　rank_idごとのamount_kgの合計を出さないといけない？？
# Calculate the average recycling amount for each user and each rank
sum_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()
# avg_amount_per_user_rank = df_merge.groupby(['user_id', 'rank_id'])['amount_kg'].sum().reset_index()

# To compare the average recycling amount of different ranks for each user, we need to pivot the data
pivot_table = sum_amount_per_user_rank.pivot(index='user_id', columns='rank_id', values='amount_kg')

# Calculate the difference between each rank's average recycling amount for each user
# For simplicity, we'll calculate the difference between consecutive ranks (rank N and rank N-1)
rank_differences = pivot_table.diff(axis=1)

In [ ]:
sum_difference_rank = rank_differences.mean(skipna=True)
sum_difference_rank